In [26]:
'''
II2202 TIVNM HT20-2 Research Methodology and Scientific Writing
Dr. Henrik's DASx Group 5
Project Code
High dimensional data
Authors: Bazil Muzaffar Kotriwala, Tamas Laczik
'''

# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split

# GAM Libraries
from pygam import LinearGAM, s, f

# Interpret ML Libraries
from interpret.glassbox import ExplainableBoostingRegressor
from interpret import show
from interpret.data import Marginal
from interpret.perf import RegressionPerf

In [27]:
# Read data
df = pd.read_csv("communities_and_crime.csv")
df.drop(['state', 'county', 'community'], axis=1, inplace=True)
df = df.apply(lambda x: x.replace('?', 0))
display(df.head(3))

,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,...,0.02,0.12,0.45,0,0,0,0,0.00,0,0.67
2,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,...,0.01,0.21,0.02,0,0,0,0,0.00,0,0.43


### Exploring the Data

In [28]:
def check_data_dimensionality(df):
    if df.shape[1] > 25:
        print('Data Dimensionality: High')
    else:
        print('Data Dimensionality: Low')

In [29]:
print('No of observations:', df.shape[0])
print('No of features:', df.shape[1])
check_data_dimensionality(df)

No of observations: 1994
No of features: 125
Data Dimensionality: High


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Columns: 125 entries, communityname to ViolentCrimesPerPop
dtypes: float64(100), int64(1), object(24)
memory usage: 1.9+ MB


In [31]:
df.describe()

,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
count,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,...,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000
mean,5.493982,0.057593,0.463395,0.179629,0.753716,0.153681,0.144022,0.424218,0.493867,0.336264,...,0.215552,0.608892,0.535050,0.626424,0.651530,0.065231,0.232854,0.161685,0.094052,0.237979
std,2.873694,0.126906,0.163717,0.253442,0.244039,0.208877,0.232492,0.155196,0.143564,0.166505,...,0.231134,0.204329,0.181352,0.200521,0.198221,0.109459,0.203092,0.229055,0.240328,0.232985
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.010000,0.350000,0.020000,0.630000,0.040000,0.010000,0.340000,0.410000,0.250000,...,0.060000,0.470000,0.420000,0.520000,0.560000,0.020000,0.100000,0.020000,0.000000,0.070000
50%,5.000000,0.020000,0.440000,0.060000,0.850000,0.070000,0.040000,0.400000,0.480000,0.290000,...,0.130000,0.630000,0.540000,0.670000,0.700000,0.040000,0.170000,0.070000,0.000000,0.150000
75%,8.000000,0.050000,0.540000,0.230000,0.940000,0.170000,0.160000,0.470000,0.540000,0.360000,...,0.280000,0.777500,0.660000,0.770000,0.790000,0.070000,0.280000,0.190000,0.000000,0.330000
max,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
# Check for null values
df[df.isnull().any(axis=1)]

,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop


### Visualizing the data

## Building a Linear GAM

In [60]:
# Prepare the data
X = df.drop(['communityname', 'ViolentCrimesPerPop'], axis=1)
X = X.apply(lambda x: pd.to_numeric(x)).values
y = df['ViolentCrimesPerPop']

# Train the model
lams = np.random.rand(100, 123)
lams = lams * 11 - 3
lams = np.exp(lams)
print(lams.shape)
gam = LinearGAM(n_splines=10).gridsearch(X, y, lam=lams)

N/A% (0 of 100) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

(100, 123)


100% (100 of 100) |######################| Elapsed Time: 0:17:59 Time:  0:17:59


In [61]:
gam.summary()

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                    383.5705
Link Function:                     IdentityLink Log Likelihood:                                 -48017.226
Number of Samples:                         1994 AIC:                                             96803.593
                                                AICc:                                           96987.9704
                                                GCV:                                                0.0223
                                                Scale:                                              0.0147
                                                Pseudo R-Squared:                                   0.7804
Feature Function                  Lam

C:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  """Entry point for launching an IPython kernel.


In [66]:
# Prepare the data
X = df.drop(['communityname', 'ViolentCrimesPerPop'], axis=1)
X = X.apply(lambda x: pd.to_numeric(x))
y = df['ViolentCrimesPerPop']

# Split the data into train and test data:
seed = 1
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

# Train the model
ebr = ExplainableBoostingRegressor()
ebr.fit(X_train, Y_train)

ExplainableBoostingRegressor(binning='quantile', early_stopping_rounds=50,
                             early_stopping_tolerance=0.0001,
                             feature_names=['fold', 'population',
                                            'householdsize', 'racepctblack',
                                            'racePctWhite', 'racePctAsian',
                                            'racePctHisp', 'agePct12t21',
                                            'agePct12t29', 'agePct16t24',
                                            'agePct65up', 'numbUrban',
                                            'pctUrban', 'medIncome', 'pctWWage',
                                            'pctWFarmSelf', 'pctWInvInc',
                                            'pctWS...
                                            'continuous', 'continuous',
                                            'continuous', 'continuous',
                                            'continuous', 'continuous',


In [67]:
# Generate local explanations
ebr_local = ebr.explain_local(X_test, Y_test)

def get_feature_local_explanations(ebr_local, X_test):
    L = []
    L_JSON = []
    for i in range(len(X_test)):
        
        # List
        feature_names = ebr_local.data(i)['names']
        local_scores = ebr_local.data(i)['scores']
        intercept = ebr_local.data(i)['extra']['scores'][0]
        actual_score = ebr_local.data(i)['perf']['actual']
        predicted_score = ebr_local.data(i)['perf']['predicted']
        difference = abs(actual_score - predicted_score)
        L.append([feature_names, local_scores, intercept, actual_score, predicted_score, difference])
        
        # json
        j = {}
        j['feature_names'] = feature_names
        j['local_scores'] = local_scores
        j['intercept'] = intercept
        j['actual_score'] = actual_score
        j['predicted_score'] = predicted_score
        j['difference'] = difference
    
        L_JSON.append(j)
        
    return L, L_JSON
    
L, L_JSON = get_feature_local_explanations(ebr_local, X_test)
print(L[0:1])
print(L_JSON[0:1])

[[['fold', 'population', 'householdsize', 'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29', 'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc', 'whitePerCap', 'blackPerCap', 'indianPerCap', 'AsianPerCap', 'OtherPerCap', 'HispPerCap', 'NumUnderPov', 'PctPopUnderPov', 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom', 'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent', 'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8', 'PctRecImmig10', 'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam', 

### Visualize GAM Local explanation

In [68]:
import json
with open('hd_data_gam_local_output.json', 'w', encoding='utf-8') as f:
    json.dump(L_JSON, f, ensure_ascii=False)

In [363]:
intercept = ebr_local.data(0)['extra']['scores']
local_scores = sum(ebr_local.data(0)['scores'])

local_scores + intercept

array([30.3404143])